In [18]:
import numpy as np
import math
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
pauli_i = [I, 0]
pauli_x = [X, 1]
pauli_y = [Y, 2]
pauli_z = [Z, 3]

def get_basis_set(num_of_qubit = 1):
    basis_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    int_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    temp_set = []
    for i in range (num_of_qubit-1):
        for j in range (len(basis_set)):
            for k in range (len(int_set)):
                a = basis_set[j]
                b = int_set[k]
                #temp_set.append()
                temp_set.append([np.kron(a[0],b[0]),[a[1],b[1]]])
        int_set = temp_set
        #print(int_set)
        temp_set = []
    return int_set

def get_decomposition(A, num_of_q):
    basis = get_basis_set(num_of_q)
    dec_list = []
    for i in range(len(basis)):
        b = basis[i]
        coeff = np.trace(A*b[0])/(2**num_of_q)
        dec_list.append([coeff, b[0], b[1]])
    return dec_list
def get_arbitary_decomposition(A):
    size = A.shape
    size = size[0]
    power = math.log2(size)
    p = int(power)
    if (p != power):
        p += 1
    ret = np.zeros([2**p,2**p], dtype = float)
    ret[:size, :size] = A
    for i in range(size, 2**p):
        ret[i,i] = 1
    return get_decomposition(ret, p)
a = np.matrix('1 2 3 4; 2 5 1 3; 3 1 6 2; 4 4 2 7')
decomp = get_arbitary_decomposition(a)
np.conj(decomp[8][0])

0.25j

In [20]:
import numpy as np
from scipy.optimize import minimize
import math

#a = a/np.linalg.norm(a)
#a = a/10
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
zKet = np.asmatrix('1;0') #np.asmatrix('1;0')
H = (1/math.sqrt(2))*np.asmatrix('1 1;1 -1')
U = np.asmatrix('0.7071 0 0 0;0.7071 0.7071 0 0; 0.7071 0 0.7071 0; 0.7071 0 0 0.7071')
#U = U/np.linalg.norm(a)
#a = a/np.linalg.norm(a)
U = U/10
a = a/10
#U = U/10
zKet = np.kron(zKet,zKet)
print(U*zKet)
lr = 0.001
#print(U)
ans = np.linalg.inv(a)*U*zKet
print((ans)/np.linalg.norm(ans))
z = np.asmatrix('1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 -1')
#calculate cost
def calculate_cost(params):
    alpha = params[0]
    alpha2 = params[2]
    alpha3 = params[1]
    alpha4 = params[3]
    Ry1 = np.matrix([[np.cos(alpha/2.), -1*np.sin(alpha/2.)], [np.sin(alpha/2.), np.cos(alpha/2.)]])
    Ry2 = np.matrix([[np.cos(alpha3/2.), -1*np.sin(alpha3/2.)], [np.sin(alpha3/2.), np.cos(alpha3/2.)]])
    Ry = np.kron(Ry1,Ry2)
    Ry3 = np.matrix([[np.cos(alpha2/2.), -1*np.sin(alpha2/2.)], [np.sin(alpha2/2.), np.cos(alpha2/2.)]])
    Ry4 = np.matrix([[np.cos(alpha4/2.), -1*np.sin(alpha4/2.)], [np.sin(alpha4/2.), np.cos(alpha4/2.)]])
    Ry5 = np.kron(Ry3,Ry4)
    V = Ry*z*Ry5
    cparam1, cparam2 = 0+0j, 0+0j
    for i in range(len(decomp)):
        for j in range(len(decomp)):
            al = decomp[i][1]
            ald = decomp[j][1]
            param1 = 0
            param1 = np.transpose(zKet)@np.transpose(U)@al@V@zKet
            param1 = param1*np.transpose(zKet)@np.transpose(V)@ald.H@U@zKet
            param1 *= decomp[i][0] * np.conj(decomp[j][0])
            cparam1 += param1 
            param2 = 0
            param2 = np.transpose(zKet) * np.transpose(V) * ald.H * al * V * zKet
            param2 *= decomp[i][0] * np.conj(decomp[j][0])
            cparam2 += param2 
    cost = 1 - (cparam1/cparam2)
    cost = np.asarray(cost)
    cost = cost[0][0]
    print("cost = " + str(cost))
    return abs(cost)


'''alpha = np.pi/6;
alpha1 = np.pi/8;
cost = calculate_cost(alpha, alpha1)'''
maxiter = 2000000
iters = 0 
threshold = 0.00000000000047
'''while((abs(cost) > threshold) and (maxiter > iters)):
    #print('##################################ITERATION - '+ str(iters) +'  ###############################')
    alpha, alpha1 = optimize(alpha, alpha1)
    cost = calculate_cost(alpha, alpha1)
    #print('##################################  END   ###############################\n')
    iters +=1
print(cost)
print(alpha,alpha1)
print(iters)
alpha2 = alpha1'''
from scipy.optimize import minimize
init_params = [np.pi/8, np.pi/4, np.pi/2, np.pi/16]
print(init_params)
calculate_cost(init_params)

out = minimize(calculate_cost, x0=init_params, method="COBYLA", options= {'maxiter':20000, 'disp': True}, tol = threshold)
print(out)
alpha = out['x'][0]
alpha2 = out['x'][2]
alpha3 = out['x'][1]
alpha4 = out['x'][3]
Ry1 = np.matrix([[np.cos(alpha/2.), -1*np.sin(alpha/2.)], [np.sin(alpha/2.), np.cos(alpha/2.)]])
Ry2 = np.matrix([[np.cos(alpha3/2.), -1*np.sin(alpha3/2.)], [np.sin(alpha3/2.), np.cos(alpha3/2.)]])
Ry = np.kron(Ry1,Ry2)
Ry3 = np.matrix([[np.cos(alpha2/2.), -1*np.sin(alpha2/2.)], [np.sin(alpha2/2.), np.cos(alpha2/2.)]])
Ry4 = np.matrix([[np.cos(alpha4/2.), -1*np.sin(alpha4/2.)], [np.sin(alpha4/2.), np.cos(alpha4/2.)]])
Ry5 = np.kron(Ry3,Ry4)
V = Ry*z*Ry5
print (V*zKet)


[[0.07071]
 [0.07071]
 [0.07071]
 [0.07071]]
[[-0.41702883]
 [ 0.62554324]
 [ 0.62554324]
 [ 0.20851441]]
[0.39269908169872414, 0.7853981633974483, 1.5707963267948966, 0.19634954084936207]
cost = (0.9810312163347243+0j)
cost = (0.9810312163347243+0j)
cost = (0.9815150495441854+0j)
cost = (0.9807939478852665+0j)
cost = (0.9810858747325021+0j)
cost = (0.9802209870019555+0j)
cost = (0.9802038441097516+0j)
cost = (0.9805181408508926+0j)
cost = (0.9801018323310167+0j)
cost = (0.9811785260529046+0j)
cost = (0.9802100804637005+0j)
cost = (0.9800613879393755+0j)
cost = (0.9801642583132737+0j)
cost = (0.98000181722122+0j)
cost = (0.9801309236515501+0j)
cost = (0.9800187455972957+0j)
cost = (0.9801206196307799+0j)
cost = (0.9800310366202213+0j)
cost = (0.9800166286456775+0j)
cost = (0.9800069876628105+0j)
cost = (0.9800059087287819+0j)
cost = (0.9800023049678499+0j)
cost = (0.9800013285087918+0j)
cost = (0.9800036243589645+0j)
cost = (0.980001942925834+0j)
cost = (0.9800015505202049+0j)
cost = (